In [120]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ape import chain, networks, Contract
import pandas as pd
import altair as alt
alt.renderers.enable('mimetype')

from helpers.aura import weekly_emissions_after_fee, aura_mint_ratio, BALANCER_EMISSIONS
from helpers.coingecko import get_prices, get_coll_prices
from helpers.formatter import pct_format, dollar_format
from helpers.bribe_info import GAUGE_CAP

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [41]:
# Locked vlAURA in voter over time
BLOCK_JUMP = 7160 * 2
deployment_block = 14932220
latest_block = networks.active_provider.get_block("latest").number
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_history = []
for block in range(deployment_block, latest_block, BLOCK_JUMP):
    timestamp = chain.blocks._get_block(block_id=int(block)).timestamp
    total_supply = vlAURA.totalSupply(block_identifier=block)/1e18
    voter_msig_power = vlAURA.getVotes('0xA9ed98B5Fb8428d68664f3C5027c62A10d45826b', block_identifier=block) /1e18
    if voter_msig_power > 0:
        ownership = voter_msig_power / total_supply
    else:
        ownership = 0
    vlAURA_history.append((timestamp, total_supply, voter_msig_power, ownership))

df = pd.DataFrame(vlAURA_history, columns = ['ts', 'total_supply', 'voter_vp', 'ownership'])
df["Date"] = pd.to_datetime(df["ts"], unit="s", utc=True)

In [121]:
# Plot the historical vlAURA pd
base = (
    alt.Chart(df.reset_index(), width=900, title=f'AURA Voting Power Insights')
    .encode(
        alt.X('Date:T', axis=alt.Axis(format='%Y-%m', labelFlush=False, tickCount=alt.TimeInterval('month')))
    )
)
total_supply = (
    base.mark_line(color='rgb(217, 95, 2)')
    .encode(
        alt.Y('total_supply', axis=alt.Axis(title='vlAURA Total Supply', titleColor='rgb(217, 95, 2)'), scale=alt.Scale(zero=False)),
    )
)
voter_vp = (
    base.mark_line(color='rgb(220,20,60)')
    .encode(
        alt.Y('voter_vp', axis=alt.Axis(title='DAO Voting Power', titleColor='rgb(220,20,60)'), scale=alt.Scale(zero=False)),
    )
)
chart = alt.layer(total_supply, voter_vp).resolve_scale(y='independent')
chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [122]:
# Ownership
base = (
    alt.Chart(df.reset_index(), width=900, title=f'Historical Ownership')
    .encode(
        alt.X('Date:T', axis=alt.Axis(format='%Y-%m', labelFlush=False, tickCount=alt.TimeInterval('month')))
    )
)
ownership = (
    base.mark_bar(width=2, color='rgb(102,205,170)')
    .encode(
        alt.Y('ownership', axis=alt.Axis(title='DAO Ownership', titleColor='rgb(102,205,170)')),
    )
)
chart = alt.layer(ownership)
chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [45]:
# tokens pricings
bal_price, aura_price,  badger_price, rpl_price = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price), "${:.2f}".format(rpl_price)]]
headers = ["Badger price", "Balancer price", "Aura price", "Rpl price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price,Rpl price
0,$2.67,$6.63,$2.00,$31.98


In [46]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, latest_block)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.58057,$2999572.8842


In [49]:
# Mint ratio progress in past months
mint_ratio_history = []
for block in range(deployment_block, latest_block, BLOCK_JUMP):
    ratio_at_block = aura_mint_ratio(aura, block)
    mint_ratio_history.append(ratio_at_block)
df_mint = pd.DataFrame(mint_ratio_history, columns = ['mint_ratio'])
df_mint['Date'] = df["Date"]

In [123]:
# Plot minting decay progress over time
base = (
    alt.Chart(df_mint.reset_index(), width=900, title=f'Minting Ratio Historical')
    .encode(
        alt.X('Date:T', axis=alt.Axis(format='%Y-%m', labelFlush=False, tickCount=alt.TimeInterval('month')))
    )
)
mint_ratio = (
    base.mark_line(color='rgb(217, 95, 2)')
    .encode(
        alt.Y('mint_ratio', axis=alt.Axis(title='Aura Mint Balancer Ratio', titleColor='rgb(217, 95, 2)'), scale=alt.Scale(zero=False)),
    )
)
chart = alt.layer(mint_ratio)
chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [53]:
# ETH expectations to be used as coll
ETH_AVAIL = 1534 # NOTE: from portfolio report
MONTHLY_BURN = 10
TRANCHE_A = MONTHLY_BURN * 3 # NOTE: quarter consideration
TRANCHE_B = MONTHLY_BURN * 6 # NOTE: additional 6m of eth expenses
TRANCHE_C = ETH_AVAIL - TRANCHE_A - TRANCHE_B

In [54]:
eth_price, btc_price  = get_coll_prices()
min_cr = 120 # NOTE: assumes 120% min-cr
max_ltv_ratio = 100 / min_cr
safe_mint_amt = 50
cl_feed = Contract('0xdeb288F737066589598e9214E782fa5A8eD689e8')
ratio = cl_feed.latestRoundData()['answer'] / (10 ** cl_feed.decimals())
max_btc_mint = TRANCHE_C / ratio * max_ltv_ratio
cr_ratio = ((TRANCHE_C * eth_price) / (safe_mint_amt * btc_price)) * 100
data = [[TRANCHE_C, safe_mint_amt, cr_ratio, max_btc_mint]]
headers = ["Tranche C ETH", "Minted Synthetic", "Collateral Ratio", "Max mint"]
df_cr = pd.DataFrame(data, columns=headers)
df_cr

,Tranche C ETH,Minted Synthetic,Collateral Ratio,Max mint
0,1444,50,213.344441,89.3114


In [55]:
# Projection of POL - wbtc/syntethicbtc
"""
assumption here is that 2x is using 50 wbtc, 3x is using 100 wbtc
3x is roughly using 50% of the total btc available as per portfolio report, currently ~ 202
"""
TOTAL_POL = safe_mint_amt * 3 * btc_price
POOL_TOTAL_LIQUIDITY = 10_000_000 # expressed in $, expected liq given incentives barely easy imo
pool_capture = TOTAL_POL / POOL_TOTAL_LIQUIDITY

In [91]:
# Aura historical veBAL % controlled
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ownership = []
for block in range(deployment_block, latest_block, BLOCK_JUMP):
    vebal_ts = vebal.totalSupplyAt(block) / 1e18
    vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", latest_block) / 1e18
    if vebal_aura_controlled > 0:
        pct_aura_vebal_controlled = vebal_aura_controlled / vebal_ts
    else:
        pct_aura_vebal_controlled = 0
    vebal_ownership.append(pct_aura_vebal_controlled)
df_aura_vebal = pd.DataFrame(vebal_ownership, columns = ['vebal_ownership'])
df_aura_vebal['Date'] = df["Date"]

In [124]:
# Plot minting decay progress over time
df_aura_vebal = df_aura_vebal[df_aura_vebal['Date'] >= '2022-08-15'] # NOTE: noise sample? focus from mid-August
base = (
    alt.Chart(df_aura_vebal.reset_index(), width=900, title=f'AURA veBAL % Historical')
    .encode(
        alt.X('Date:T', axis=alt.Axis(format='%Y-%m', labelFlush=False, tickCount=alt.TimeInterval('month')))
    )
)
ownership_vebal = (
    base.mark_bar(width=2, color='rgb(102,205,170)')
    .encode(
        alt.Y('vebal_ownership', axis=alt.Axis(title='veBAL Ownership', titleColor='rgb(102,205,170)', format='%')),
    )
)
chart = alt.layer(ownership_vebal)
chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [105]:
vebal_ts = vebal.totalSupplyAt(latest_block) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", latest_block) / 1e18
pct_aura_vebal_controlled = vebal_aura_controlled / vebal_ts
pct_badger_vebal_controlled  = df['ownership'].iat[-1] * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled
0,23.541%,0.609%


In [74]:
# Cost for reaching caps, without external actors self-incentive support
MULTIPLIER = 1 # NOTE: tinker multiplier for 4%, 6%, 8%, 10% gauges 
GAUGE_CAP = GAUGE_CAP * MULTIPLIER
proactive_vebal_voters = 10693175.86
cost_per_vote_vebal = (BALANCER_EMISSIONS * bal_price) / proactive_vebal_voters
vebal_controlled = vebal_ts * pct_badger_vebal_controlled
vebal_max_cap = vebal_ts * (GAUGE_CAP)
vebal_required = vebal_max_cap - vebal_controlled
cost_max_cap = vebal_required * cost_per_vote_vebal
badger_to_incentive = cost_max_cap / badger_price
data = [[vebal_max_cap, vebal_controlled, dollar_format(cost_max_cap), badger_to_incentive]]
headers = ["veBAL pool Max Cap", "veBAL BADGER controlled", "$ Cost of incentive", "Badger Incentive Max Cap"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,veBAL pool Max Cap,veBAL BADGER controlled,$ Cost of incentive,Badger Incentive Max Cap
0,225700.164741,68757.463298,$14109.6404,5284.50951


In [78]:
# treasury expected earnings
pool_earnings = biweekly_emissions * GAUGE_CAP
treasury_roi = (pool_earnings*pool_capture) / cost_max_cap
data = [[dollar_format(pool_earnings), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "Pool Capture - Projection", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_pool = pd.DataFrame(data, columns=headers)
df_earnings_pool

,Pool yield (USD) - biweekly,Pool Capture - Projection,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$59991.4577,31.869%,$19118.6776,135.501%
